In [15]:
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT, force_remount=True)           # we mount the google drive at /content/drive

/content/drive
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [18]:
%cd 'drive/My Drive/Workspaces/machinelearning/Projects/Udacity MLND/Capstone Project/'
# %pwd

/content/drive/My Drive/Workspaces/machinelearning/Projects/Udacity MLND/Capstone Project


In [5]:
# !pip install textstat
# !pip install pyLDAvis

     |████████████████████████████████| 1.6MB 2.8MB/s 
     |████████████████████████████████| 552kB 16.6MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=e6820087d528e02582a20febc514c10c43d61cacfc36794404014380a577bb69
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=fe1347f76a0de93ea5922f91fde451ea9b8d0666a56387f9d133765ece245096
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy


In [25]:
# Imports
import string
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

%matplotlib inline

from plotly import tools
import plotly.offline as py
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

from sklearn import model_selection, preprocessing, metrics, ensemble, naive_bayes, linear_model
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from statistics import *
import concurrent.futures
import time
import pyLDAvis.sklearn
from pylab import bone, pcolor, colorbar, plot, show, rcParams, savefig
import textstat

import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

import pandas as pd
pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999

import numpy as np

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import folium 
from folium import plugins 

import re


In [9]:

#utility functions:
def plot_readability(a,b,title,bins=0.1,colors=['#3A4750', '#F64E8B']):
  trace1 = ff.create_distplot([a,b], [" Real disaster tweets","Not real disaster tweets"], bin_size=bins, colors=colors, show_rug=False)
  
  trace1['layout'].update(title=title)
  py.iplot(trace1, filename='Distplot')
  table_data= [["Statistical Measures"," Not real disaster tweets","real disaster tweets"], ["Mean",mean(a),mean(b)], ["Standard Deviation",pstdev(a),pstdev(b)],
               ["Variance",pvariance(a),pvariance(b)],
               ["Median",median(a),median(b)],
               ["Maximum value",max(a),max(b)],
               ["Minimum value",min(a),min(b)]]
  trace2 = ff.create_table(table_data)
  py.iplot(trace2, filename='Table')

punctuations = string.punctuation
stopwords = list(STOP_WORDS)

parser = English()
def spacy_tokenizer(sentence):
    mytokens = parser(sentence)
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    mytokens = [ word for word in mytokens if word not in stopwords and word not in punctuations ]
    mytokens = " ".join([i for i in mytokens])
    return mytokens


def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext

def removeurl(raw_text):
  clean_text = re.sub(r'^https?:\/\/.*[\r\n]*', '', raw_text, flags=re.MULTILINE)
  return clean_text

### Import Dataset


In [28]:
# Importing cleaned data
train_df = pd.read_csv('data/preprocessed_train.csv')
test_df = pd.read_csv('data/preprocessed_test.csv')
sub_df = pd.read_csv('data/sample_submission.csv')


In [29]:
train_df.head()

,Unnamed: 0,id,keyword,location,text,target,word_count,unique_word_count,stopword_count,url_count,mean_word_length,char_count,punctuation_count,hashtag_count,mention_count,text_cleaned,target_relabeled
0,0,1,no_keyword,no_location,Our Deeds are the Reason of this #earthquake M...,1,13,13,5,0,4.384615,69,1,1,0,Our Deeds are the Reason of this # earthquake...,1
1,1,4,no_keyword,no_location,Forest fire near La Ronge Sask. Canada,1,7,7,0,0,4.571429,38,1,0,0,Forest fire near La Ronge Sask . Canada,1
2,2,5,no_keyword,no_location,All residents asked to 'shelter in place' are ...,1,22,20,9,0,5.090909,133,3,0,0,All residents asked to ' shelter in place ' ...,1
3,3,6,no_keyword,no_location,"13,000 people receive #wildfires evacuation or...",1,8,8,1,0,7.125000,65,2,1,0,"13,000 people receive # wildfires evacuation ...",1
4,4,7,no_keyword,no_location,Just got sent this photo from Ruby #Alaska as ...,1,16,15,6,0,4.500000,88,2,2,0,Just got sent this photo from Ruby # Alaska a...,1


### Topic Modeling

In [40]:
notreal_text = train_df[train_df["target"] == 0]['text_cleaned'].apply(spacy_tokenizer)
real_text = train_df[train_df["target"] == 1]['text_cleaned'].apply(spacy_tokenizer)
#count vectorization
vectorizer_notreal = CountVectorizer(min_df=5, max_df=0.9, stop_words='english', lowercase=True, token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
notreal_vectorized = vectorizer_notreal.fit_transform(notreal_text)
vectorizer_real = CountVectorizer(min_df=5, max_df=0.9, stop_words='english', lowercase=True, token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
real_vectorized = vectorizer_real.fit_transform(real_text)

#### Latent Dirichlet Allocation (LDA)

Latent Dirichlet Allocation (LDA) and LSA are based on the same underlying assumptions: the distributional hypothesis, (i.e. similar topics make use of similar words) and the statistical mixture hypothesis (i.e. documents talk about several topics) for which a statistical distribution can be determined. The purpose of LDA is mapping each document in our corpus to a set of topics which covers a good deal of the words in the document.

What LDA does in order to map the documents to a list of topics is assign topics to arrangements of words, e.g. n-grams such as best player for a topic related to sports. This stems from the assumption that documents are written with arrangements of words and that those arrangements determine topics.LDA ignores syntactic information and treats documents as bags of words. It also assumes that all words in the document can be assigned a probability of belonging to a topic. That said, the goal of LDA is to determine the mixture of topics that a document contains.

In other words, LDA assumes that topics and documents look like this:

In [41]:
# Latent Dirichlet Allocation Model
lda_notreal = LatentDirichletAllocation(n_components=10, max_iter=5, learning_method='online',verbose=True)
notreal_lda = lda_notreal.fit_transform(notreal_vectorized)
lda_real = LatentDirichletAllocation(n_components=10, max_iter=5, learning_method='online',verbose=True)
real_lda = lda_real.fit_transform(real_vectorized)

iteration: 1 of max_iter: 5
iteration: 2 of max_iter: 5
iteration: 3 of max_iter: 5
iteration: 4 of max_iter: 5
iteration: 5 of max_iter: 5
iteration: 1 of max_iter: 5
iteration: 2 of max_iter: 5
iteration: 3 of max_iter: 5
iteration: 4 of max_iter: 5
iteration: 5 of max_iter: 5


In [42]:
def selected_topics(model, vectorizer, top_n=10):
  for idx, topic in enumerate(model.components_):
    print("Topic %d:" % (idx))
    print([(vectorizer.get_feature_names()[i], topic[i])
    for i in topic.argsort()[:-top_n - 1:-1]]) 

In [43]:
print("Not real disaster tweets LDA Model:")
selected_topics(lda_notreal, vectorizer_notreal)

Not real disaster tweets LDA Model:
Topic 0:
[('way', 58.06277722276773), ('content', 55.82648538575393), ('best', 55.66115129679637), ('black', 48.9427772741777), ('going', 48.56164723417541), ('smoke', 45.455234580553515), ('burning', 40.175680977326984), ('movie', 35.310250955507286), ('summer', 34.34874590059112), ('long', 33.52409840208277)]
Topic 1:
[('think', 69.85263965300277), ('time', 48.62606834253217), ('right', 45.65395151912937), ('come', 38.809194577249265), ('help', 38.569656005608046), ('face', 38.07525480430576), ('disaster', 32.14121444501833), ('lot', 31.128040992020917), ('said', 30.79235799913124), ('things', 30.15316676008208)]
Topic 2:
[('new', 177.4449846122662), ('love', 101.64625869665542), ('wreck', 72.60818638491881), ('reddit', 65.78361988389202), ('good', 47.66886905198804), ('work', 39.60238430226646), ('wrecked', 34.46230037379804), ('car', 32.898157581635935), ('thank', 29.608158363931278), ('soon', 27.960066446151686)]
Topic 3:
[('know', 83.1337456315

In [44]:
print("Real disaster tweets LDA Model:")
selected_topics(lda_real, vectorizer_real)

Real disaster tweets LDA Model:
Topic 0:
[('wreckage', 100.40241825966359), ('typhoon', 73.49314011252169), ('oil', 50.08295500954981), ('warning', 50.00547138494818), ('earthquake', 49.0946280122255), ('devastated', 34.8907026328124), ('severe', 33.56193681691898), ('soudelor', 31.989228919697617), ('massacre', 31.917537855092817), ('declares', 29.56179488673975)]
Topic 1:
[('storm', 97.07496550052961), ('train', 73.84778207693314), ('dead', 65.50040611411607), ('migrants', 45.855000163569514), ('thunderstorm', 38.82810132971095), ('calgary', 37.52859350889785), ('hundreds', 35.844699030389364), ('violent', 33.890316844133295), ('derailment', 32.83233842430505), ('charged', 31.672060887302102)]
Topic 2:
[('news', 133.19783149745516), ('police', 120.9152111593028), ('attack', 69.0685530968979), ('obama', 62.471969418540226), ('fatal', 56.103286252262585), ('latest', 52.90777596255594), ('mass', 51.528949045641504), ('world', 45.45102484777986), ('terrorist', 39.48927961461956), ('offic

##### Visualizing LDA results of not a real disaster tweets with pyLDAvis

In [45]:
pyLDAvis.enable_notebook()
dash = pyLDAvis.sklearn.prepare(lda_notreal, notreal_vectorized, vectorizer_notreal, mds='tsne')
dash

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
9      32.420952   8.115230       1        1  10.633531
0       0.648955  71.707100       2        1  10.619295
4      71.019684   9.971006       3        1  10.387583
8       3.785981  33.185135       4        1  10.268712
5     -42.666500  -9.222070       5        1  10.139411
6      -3.621689 -43.526516       6        1  10.042807
2      -3.632402  -4.171708       7        1   9.790678
3     -36.584400  35.804413       8        1   9.628094
1      44.391380  50.615383       9        1   9.268146
7      40.235809 -31.591665      10        1   9.221741, topic_info=        Term        Freq       Total Category  logprob  loglift
720     like  249.000000  249.000000  Default  30.0000  30.0000
839      new  159.000000  159.000000  Default  29.0000  29.0000
521      got   88.000000   88.000000  Default  28.0000  28.0000
753     love   92.000000   92.000000  Default  27.0000  27.0000
132     body   88.000000   88.000000  Default  26.0000  26.0000
...      ...         ...         ...      ...      ...      ...
656     isis   11.699338   12.542345  Topic10  -5.0530   2.3140
209    check   29.313240   34.156043  Topic10  -4.1345   2.2307
768      man   47.338216   59.773390  Topic10  -3.6552   2.1504
1337  weapon   23.840871   37.129137  Topic10  -4.3411   1.9406
1178    sunk   23.280261   37.930962  Topic10  -4.3649   1.8954

[400 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
10        8  0.925697  actually
12        6  0.919181     added
22       10  0.952862     album
28        4  0.938568   america
30        1  0.935670   ancient
...     ...       ...       ...
1391     10  0.196391      year
1392      9  0.959298     years
1393     10  0.970615       yes
1396      6  0.985456   youtube
1397      3  0.968602      zone

[477 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 1, 5, 9, 6, 7, 3, 4, 2, 8])

##### Visualizing LDA results of a real disaster tweets with pyLDAvis

In [46]:
pyLDAvis.enable_notebook()
dash = pyLDAvis.sklearn.prepare(lda_real, real_vectorized, vectorizer_real, mds='tsne')
dash

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
2      32.420952   8.115230       1        1  12.739369
5       0.648955  71.707100       2        1  11.382530
9      71.019684   9.971006       3        1  10.757512
4       3.785981  33.185135       4        1  10.731804
1     -42.666500  -9.222070       5        1  10.336225
3      -3.621689 -43.526516       6        1  10.163857
7      -3.632402  -4.171708       7        1   9.482256
0     -36.584400  35.804413       8        1   8.537356
6      44.391380  50.615383       9        1   8.085891
8      40.235809 -31.591665      10        1   7.783201, topic_info=               Term        Freq       Total Category  logprob  loglift
635        malaysia  171.000000  171.000000  Default  30.0000  30.0000
148      california  119.000000  119.000000  Default  29.0000  29.0000
295        disaster  115.000000  115.000000  Default  28.0000  28.0000
391          flight  118.000000  118.000000  Default  27.0000  27.0000
25         airlines  112.000000  112.000000  Default  26.0000  26.0000
...             ...         ...         ...      ...      ...      ...
1186           wind   15.260761   25.721874  Topic10  -4.5576   2.0311
714            near   18.675761   43.975129  Topic10  -4.3556   1.6968
93    bestnaijamade   14.457526   26.952399  Topic10  -4.6116   1.9303
327       emergency   18.154662   62.667040  Topic10  -4.3839   1.3143
337       evacuated   13.396440   28.279028  Topic10  -4.6879   1.8061

[404 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
2         2  0.973213       abc
4         3  0.835188  accident
4        10  0.122223  accident
13        7  0.965664  affected
25        2  0.989424  airlines
...     ...       ...       ...
1206      8  0.990717  wreckage
1211      4  0.887395      year
1211      5  0.098599      year
1212      4  0.984382     years
1214      9  0.955552      york

[473 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 6, 10, 5, 2, 4, 8, 1, 7, 9])

#### TFIDF Vectorization

In [66]:
tfidf_vec = TfidfVectorizer(stop_words='english', ngram_range=(1,3))
tfidf_vec.fit_transform(train_df['text_cleaned'].values.tolist() + test_df['text_cleaned'].values.tolist())
train_tfidf = tfidf_vec.transform(train_df['text_cleaned'].values.tolist())
test_tfidf = tfidf_vec.transform(test_df['text_cleaned'].values.tolist())

#### Building Classification models

##### 1. Logistic Regression

In [69]:
train_y = train_df.target.values

def runModel(train_X, train_y, test_X, test_y, test_X2):
  model = linear_model.LogisticRegression(C=5., solver='sag')
  model.fit(train_X, train_y)
  pred_test_y = model.predict_proba(test_X)[:,1]
  pred_test_y2 = model.predict_proba(test_X2)[:,1]
  return pred_test_y, pred_test_y2, model

print("Building model.")

cv_scores = []
pred_full_test = 0
pred_train = np.zeros([train_df.shape[0]])
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=8)
for dev_index, val_index in kf.split(train_df):
  dev_X, val_X = train_tfidf[dev_index], train_tfidf[val_index]
  dev_y, val_y = train_y[dev_index], train_y[val_index]
  pred_val_y, pred_test_y, model = runModel(dev_X, dev_y, val_X, val_y, test_tfidf)
  pred_full_test = pred_full_test + pred_test_y
  pred_train[val_index] = pred_val_y
  cv_scores.append(metrics.log_loss(val_y, pred_val_y))
  break

Building model.


In [70]:
from tqdm import tqdm

def threshold_search(y_true, y_proba):
  #reference: https://www.kaggle.com/hung96ad/pytorch-starter
  best_threshold = 0
  best_score = 0
  for threshold in tqdm([i * 0.001 for i in range(1000)]):
    score = metrics.f1_score(y_true=y_true, y_pred=y_proba > threshold)
    if score > best_score:
      best_threshold = threshold
      best_score = score
      search_result = {'threshold': best_threshold, 'f1': best_score}
      return search_result
      
search_result = threshold_search(val_y, pred_val_y)
search_result

  0%|          | 0/1000 [00:00<?, ?it/s]


{'f1': 0.6065873741994511, 'threshold': 0.0}

In [55]:
print("F1 score at threshold {0} is {1}".format(0.381, metrics.f1_score(val_y, (pred_val_y>0.381).astype(int))))
print("Precision at threshold {0} is {1}".format(0.381, metrics.precision_score(val_y, (pred_val_y>0.381).astype(int))))
print("recall score at threshold {0} is {1}".format(0.381, metrics.recall_score(val_y, (pred_val_y>0.381).astype(int))))

F1 score at threshold 0.381 is 0.7624720774385705
Precision at threshold 0.381 is 0.7551622418879056
recall score at threshold 0.381 is 0.7699248120300752
